# Uber Prize Starter Kit Python Utilities: Tutorial

To simplify interaction with the Docker-based simulation execution and evaluation, we've provided a set of Python utilities (located in the `/utilities` folder of the Starter-Kit repository). This notebook demonstrates how they may be used to accomplish the following tasks:

 - Starting a simulation or several simulations
 - Checking simulation completion
 - Retrieving the simulation score in a convenient Pandas `DataFrame` format.
 - Generating fake data 

*Note*: It is assumed that this notebook is started from the `/examples` folder.

In [7]:

# Adding the module to the path for future import
import sys
import os
import docker
from pathlib import Path
# Note that the following is idempotent when this notebook is run from "/examples"
os.chdir('../utilities')


## The `competition_executor` Module

A `CompetitionContainerExecutor` object may be used to start, stop, and gather information about containers running simulations and/or completed simulation scores and stats.

In [9]:
from competition_executor import CompetitionContainerExecutor

path_input = r"/submission-inputs"
path_ouput = r"/output"

my_executor = CompetitionContainerExecutor(input_root=path_input,
                                  output_root=path_ouput)

# Note: Instantiating a CompetitionContainerExecutor with path arguments is not strictly necessary. Each simulation
# can be run with its own set of input and/or output arguments. However, if you prefer to designate
# a single directory for inputs and/or a single directory for outputs, then, for convenience, 
# you may pass those arguments in here and avoid re-entering them for each simulation run.

Run the simulation using the `run_sumulation` method. For example:

In [ ]:
# Note: the `submission_id`s for each run must be unique.
try:
    my_submission = my_executor.client.containers.get('my_submission')
    my_submission.stop()
    my_submission.remove()
    print("Found container from previous run, removing and creating new sim container...")
except docker.errors.NotFound:
    print("Creating new simulation container...")
    
my_executor.run_simulation('my_submission', num_iterations=1, num_cpus=10)

# Note that a dictionary indicating which containers IDs already exist can be accessed.
for key, value in ex.containers.items():
    print("Container ID : {0}".format(key))

You may view the **logs** of a specific simulation as follows:

In [ ]:
logs = my_executor.output_simulation_logs('my_submission')

Submissions run for a certain amount of time. If you interrupt it before the end, you will not get any outputs. You can **check if a submission is finished** with the following method:

In [27]:
my_executor.check_if_submission_complete('my_submission')

True

When a simulation run is done, you can import its **results**:

In [28]:
scores, stats = my_executor.get_submission_scores_and_stats('my_submission')

The **scores** and **statistics** are stored in pandas DataFrames which contains the information described [here](https://github.com/vgolfier/Uber-Prize-Starter-Kit/blob/master/docs/Understanding_the_outputs_and_the%20scoring_function.md).

In [29]:
scores

,Budget: Operational Costs (Fixed)_Weight,Budget: Operational Costs (Fuel)_Weight,"Budget: Operational Costs (Variable, Hourly)_Weight",Budget: Subsidies Paid_Weight,Budget: Subsidies Unpaid_Weight,Congestion: Vehicle Hours Delay_Weight,Congestion: Vehicle Miles Traveled_Weight,Level of Service: Agent Hours on Crowded Transit_Weight,Level of Service: Travel Expenditure_Weight,Submission Score_Weight,...,Budget: Operational Costs (Fixed)_Weighted Score,Budget: Operational Costs (Fuel)_Weighted Score,"Budget: Operational Costs (Variable, Hourly)_Weighted Score",Budget: Subsidies Paid_Weighted Score,Budget: Subsidies Unpaid_Weighted Score,Congestion: Vehicle Hours Delay_Weighted Score,Congestion: Vehicle Miles Traveled_Weighted Score,Level of Service: Agent Hours on Crowded Transit_Weighted Score,Level of Service: Travel Expenditure_Weighted Score,Submission Score_Weighted Score
0,0.001,0.001,0.001,0.001,0.001,0.01,0.0001,10.0,0.0001,0.0,...,0.0,0.0,0.0,0.0,0.0,-16.942622,-0.180887,-80.747222,-0.072772,-97.943504


In [30]:
stats

,Iteration,agentHoursOnCrowdedTransit,fuelConsumedInMJ_Diesel,fuelConsumedInMJ_Food,fuelConsumedInMJ_Gasoline,numberOfVehicles_BODY-TYPE-DEFAULT,numberOfVehicles_BUS-DEFAULT,numberOfVehicles_CAR-TYPE-DEFAULT,personTravelTime_car,personTravelTime_drive_transit,...,totalSubsidy_walk_transit,totalTravelTime,totalVehicleDelay,vehicleHoursTraveled_BODY-TYPE-DEFAULT,vehicleHoursTraveled_BUS-DEFAULT,vehicleHoursTraveled_CAR-TYPE-DEFAULT,vehicleMilesTraveled_BODY-TYPE-DEFAULT,vehicleMilesTraveled_BUS-DEFAULT,vehicleMilesTraveled_CAR-TYPE-DEFAULT,vehicleMilesTraveled_total
0,0,8.055000,130444.966235,40.656164,23822.196390,1003.0,255.0,1128.0,18183.833333,2856.083333,...,0.0,8386.926667,8481.054444,164.778056,156.916667,339.018889,476.652242,4043.033927,4048.825914,8568.512083
1,1,8.074722,130444.966235,41.843421,23736.722191,1003.0,255.0,1128.0,35490.416667,3586.200000,...,0.0,8140.282778,8236.082222,169.594444,156.916667,653.235833,490.571632,4043.033927,4034.298700,8567.904259


At the end of any run and to avoid any conflicts between submission ids (particularly if you want to reuse names), it is advised to **clean up containers** calling the following method:

In [31]:
my_executor.stop_all_simulations()